In [ ]:
import cv2
import pandas as pd
import numpy as np
import torch
import ultralytics
from ultralytics import YOLO
from tracker import*

In [ ]:
model = YOLO('models/yolo12s.pt')  # Path to the model file

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f"Using device: {device}")


cpu


In [ ]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

my_class = ['car', 'motorcycle', 'bus', 'truck', 'rickshaw', 'lorry']


In [ ]:
# Initialize counters for each vehicle type
vehicle_counts = {
    'car': 0,
    'motorcycle': 0,
    'bus': 0,
    'truck': 0,
    'rickshaw': 0,
    'lorry': 0
}

# Initialize a set to keep track of counted vehicle IDs
counted_ids = set()
# Initialize the Tracker

tracker=Tracker()
video=cv2.VideoCapture('road_feed.mp4')

In [ ]:


# Main loop to process video frames
while True:
    ret, frame = video.read()
    if not ret:
        break

    # Resize frame (optional, adjust as needed)
    frame = cv2.resize(frame, (540, 960))

    # Predict objects in the frame using YOLOv12
    results = model.predict(frame)
    temp = results[0].boxes.data
    temp = temp.detach().cpu().numpy()
    bb_df = pd.DataFrame(temp).astype("int")

    # Filter detected objects
    bb_list = []
    for index, row in bb_df.iterrows():
        x1, y1, x2, y2, sc, d = row
        c = class_list[d]
        if c in my_class:
            bb_list.append([x1, y1, x2, y2, sc, c])

    # Update tracker with detected objects
    bbox_ids = tracker.update(bb_list)

    # Draw bounding boxes and IDs on the frame
    for bbox in bbox_ids:
        x3, y3, x4, y4, sc, c, id = bbox
        cx = int(x3 + x4) // 2
        cy = int(y3 + y4) // 2

        # Draw bounding box and label
        cv2.putText(frame, str(c), (x3, y3 - 2), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1)
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 1)

        # Count vehicles crossing a line
        counter_line_y = 480
        offset = 5

        # Draw counter line
        cv2.line(frame, (150, counter_line_y), (450, counter_line_y), (0, 255, 0), 3)
        cv2.putText(frame, 'counter line', (150, counter_line_y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        # Check if vehicle crosses the counter line
        if counter_line_y < (cy + offset) and counter_line_y > (cy - offset):
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

            # Ensure each vehicle is counted only once
            if id not in counted_ids:
                counted_ids.add(id)  # Add ID to the set
                vehicle_counts[c] += 1  # Increment the count for the detected vehicle type

    # Display vehicle counts on the frame
    y_offset = 40
    for vehicle_type, count in vehicle_counts.items():
        cv2.putText(frame, f'{vehicle_type}: {count}', (60, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        y_offset += 30  # Move down for the next line

    # Display total count on the frame
    cv2.putText(frame, f'Total Vehicles: {len(counted_ids)}', (60, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Show the frame
    cv2.imshow("Vehicle Detection", frame)

    # Exit on 'ESC' key press
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release video and close windows
video.release()
cv2.destroyAllWindows()